# To Do 

1. Fix code

    1.1 Use true parameters except for delta and see if that works
    
    1.2 Upload problem to PyMC3 discussion board / there is this one guy who answers *all* questions 

# Bayesian estimation

## Import Modules

In [1]:
import respy as rp
import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt

## Create wrapper class around custom Log-Likelihood

In [55]:
class respyLogLik(tt.Op):
    """
    Class wrapper around loglikehood s.t. PyMC3 
    can use the custom likelihood function from respy.
    Inherits from tt.Op. 
    """
    
    itypes = [tt.dvector]
    otypes = [tt.dscalar] 
    
    def __init__(self, loglik, param_idx):
        """
        Initialize the Op with <loglik> function.
        
        Parameters
        ----------
        loglik:
            The log-likelihood function 
        """
        
        self.likelihood = loglik
        self.index = param_idx
    
    def perform(self, node, inputs, outputs):
        """Method which is used when calling the Op"""

        # modify input for respy log-likelihood
        theta = pd.Series(data=inputs[0].reshape(-1),
                         index=self.index)
        
        # compute log-likelihood value        
        logl = self.likelihood(theta)
        
        # store output
        outputs[0][0] = np.array(logl)

## Simulate (robinson) data 

In [49]:
theta_0, options_0, data = rp.get_example_model("robinson", with_data=True)

## Create Log-Likelihood which is only dependent on parameters (theta)

In [18]:
respy_likelihood = rp.get_crit_func(theta_0, options_0, data)
respy_likelihood(theta_0)

9.033443270309705

## Set values for MCMC simulation

In [19]:
ndraws = 10  #int(5e3)
nburn = 3  #int(1e1)

## Set Log-Likelihood

In [56]:
loglik = respyLogLik(respy_likelihood, theta_0.index)

## Define priors and sample from Log-Likelihood using ```pyMC3```

In [57]:
with pm.Model():
    
    ## Priors; we choose very large variances to make them uninformative 
    
    # discount factor is uniform on [0, 1]
    delta = pm.Uniform('delta', lower=0., upper=1.)
    
    # experience accumulation parameter is HalfNormal with very large variance
    exp_fishing = pm.HalfNormal('exp_fishing', sigma=1e3)
    
    # constant priors are normal with very large variance and centered around zero
    nonpec_fishing = pm.Normal('nonpec_fishing', mu=0., sigma=1e3)
    nonpec_hammock = pm.Normal('nonpec_hammock', mu=0., sigma=1e3)
    
    # standard deviations are InverseGamma with very large variance
    sd_fishing = pm.InverseGamma('sd_fishing', alpha=0.01, beta=0.01)
    sd_hammock = pm.InverseGamma('sd_hammock', alpha=0.01, beta=0.01)
    
    # correlation is uniform on [-1, 1]
    corr_hammock_fishing = pm.Uniform('corr_hammock_fishing', lower=-1., upper=1.)
    
    ## Parameter vector (convert parameters to tensor vector)
    theta = tt.as_tensor_variable((
        delta,
        exp_fishing,
        nonpec_fishing,
        nonpec_hammock,
        sd_fishing,
        sd_hammock,
        corr_hammock_fishing
    ))
    
    ## Density Distribution 
    pm.DensityDist('likelihood', lambda v: loglik(v), observed={'v': theta})
    
    
    ## Trace  
    trace = pm.sample(ndraws, tune=nburn)  #, discard_tuned_samples=True)

Only 10 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Slice: [corr_hammock_fishing]
>Slice: [sd_hammock]
>Slice: [sd_fishing]
>Slice: [nonpec_hammock]
>Slice: [nonpec_fishing]
>Slice: [exp_fishing]
>Slice: [delta]
Sampling 2 chains:   0%|          | 0/26 [00:27<?, ?draws/s]


ValueError: Not enough samples to build a trace.